In [13]:
# importing required pakcages
import pandas as pd
from tqdm import tqdm
import yaml

In [14]:
def read_yaml(file_path):
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

ds = read_yaml(file_path = "configs.yaml")

In [15]:
ds_config = "LastFM"

In [16]:
dataset = pd.read_csv(f"datasets/{ds[ds_config]['ds_name']}/ratings_data.txt", sep=ds[ds_config]['ds_sep'], names=ds[ds_config]['ds_names'])

In [17]:
dataset.head()

,uid,iid,rating
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [18]:
# what columns or attributes the dataset is included?
dataset.columns

Index(['uid', 'iid', 'rating'], dtype='object')

In [19]:
# statistics on explicit dataset
print("Original dataset statistics: ")
print(f"> No. of users: {len(dataset['uid'].unique())}")
print(f"> No. of Items: {len(dataset['iid'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

Original dataset statistics: 
> No. of users: 1892
> No. of Items: 17632
> No. of Interaction: 92834


In [20]:
# checking the number of times in which a rating happened - (0: implicit, 1-10: explicit)
dataset['rating'].value_counts()

1       636
2       419
4       345
3       338
5       318
       ... 
2898      1
4945      1
8983      1
5867      1
2049      1
Name: rating, Length: 5436, dtype: int64

In [21]:
# This method return a dataframe in which the specified values are removed from a specific column. We can use it to generate the implicit/explicit dataset.
# To do this, we can remove the 0 or 1-10 values from the `Book-Rating` column.
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [22]:
# To remove the users with fewer than 5 interaction we first count the number of interactino per user and add a new column (`Count`) in the dataframe.
# This column shows the number of interaction per user in the dataset
users_counts = dataset['uid'].value_counts()
users_counts = users_counts.to_dict() #converts to dictionary
dataset['count'] = dataset['uid'].map(users_counts)

### Iteratively remove users and items with fewer than `ds_rate` interactions

In [23]:
user_interaction, item_interaction = 1, 1

while user_interaction != 0 or item_interaction != 0:
    print(f"The current number of user and item with < {ds[ds_config]['ds_core']} interactions: ")
    # user side fewer than ds['ds_core'] cheking
    uid_value_counts = dataset['uid'].value_counts()
    user_interaction = uid_value_counts[uid_value_counts < ds[ds_config]['ds_core']].count()
    print(f"No. of users < {ds[ds_config]['ds_core']} ineractions: {user_interaction}")

    users_counts = dataset['uid'].value_counts()
    users_counts = users_counts.to_dict() #converts to dictionary
    dataset['count'] = dataset['uid'].map(users_counts)

    dataset = filter_rows_by_values(dataset, "count", list(range(ds[ds_config]['ds_core'])))

    # item side fewer than ds_rate cheking
    bid_value_counts = dataset['iid'].value_counts()
    item_interaction = bid_value_counts[bid_value_counts < ds[ds_config]['ds_core']].count()
    print(f"No. of items < {ds[ds_config]['ds_core']} ineractions: {item_interaction}")

    items_counts = dataset['iid'].value_counts()
    items_counts = items_counts.to_dict() #converts to dictionary
    dataset['count'] = dataset['iid'].map(items_counts)

    dataset = filter_rows_by_values(dataset, "count", list(range(ds[ds_config]['ds_core'])))

The current number of user and item with < 10 interactions: 
No. of users < 10 ineractions: 18
No. of items < 10 ineractions: 16083
The current number of user and item with < 10 interactions: 
No. of users < 10 ineractions: 68
No. of items < 10 ineractions: 20
The current number of user and item with < 10 interactions: 
No. of users < 10 ineractions: 2
No. of items < 10 ineractions: 2
The current number of user and item with < 10 interactions: 
No. of users < 10 ineractions: 0
No. of items < 10 ineractions: 0


In [24]:
# statistics on 5 rate explicit dataset (after pre-processing)
print(f"No. of users: {len(dataset['uid'].unique())}")
print(f"No. of Items: {len(dataset['iid'].unique())}")
print(f"No. of Interaction: {dataset.shape[0]}")

No. of users: 1797
No. of Items: 1507
No. of Interaction: 62376


In [25]:
# Before we save the preprocessed explicit dataset (5Rate) we first remove the added column which is `Count`
del dataset['count']
if 'timestamp' in ds[ds_config]['ds_names']:
    del dataset['timestamp']

### User and Item ID Mapping

In [26]:
uid_to_index = dict()
iid_to_index = dict()

last_user_id = 0
last_item_id = 0

for eachline in tqdm(dataset.iterrows()):
  # add a new user id with an index
  if eachline[1][0] not in uid_to_index.keys():
    uid_to_index[eachline[1][0]] = last_user_id
    last_user_id += 1
  # add a new book id with an index
  if eachline[1][1] not in iid_to_index.keys():
    iid_to_index[eachline[1][1]] = last_item_id
    last_item_id += 1

62376it [00:06, 9839.54it/s] 


In [27]:
# write the file with new mapped indices into a txt file
mapped_dataset_txt = open(f"datasets/{ds[ds_config]['ds_name']}/{ds[ds_config]['ds_name']}_data.txt", 'w')

for eachline in tqdm(dataset.iterrows()):
    mapped_dataset_txt.write(str(uid_to_index[eachline[1][0]]) + "," + str(iid_to_index[eachline[1][1]]) + "," + str(eachline[1][2]) + "\n")
mapped_dataset_txt.close()

62376it [00:07, 7940.84it/s]
